In [ ]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['font.size'] = 15
import seaborn as sns
from plotnine import *
import missingno as msno
import folium

전국도시공원정보표준데이터


다운받은 데이터를 로드하고 관찰한다.

In [ ]:
# ANSI 파일일 경우 한글이 안 읽히기 때문에 encoding 써주어야 한다. or 다른 이름 저장(UTF-8)
park_202308 = pd.read_csv('./data/park_202308.csv', encoding='cp949')
park_202308

In [ ]:
park_202308.info

In [ ]:
park_202308.isnull().sum()

In [ ]:
msno.matrix(park_202308)
plt.show()

In [ ]:
park_202308.columns

In [ ]:
# drop() 메소드로 불필요한 컬럼을 제거한다.
park_202308.drop(columns=['공원보유시설(운동시설)', '공원보유시설(유희시설)', '공원보유시설(편익시설)', '공원보유시설(교양시설)',
       '공원보유시설(기타시설)', '지정고시일', '관리기관명'], inplace=True)
park_202308

In [ ]:
(
    ggplot(park_202308, aes(x='경도', y='위도'))
        + geom_point()
        + theme(text=element_text(family='NanumGothicCoding'), figure_size=(12, 8))
)

In [ ]:
park_202308.plot.scatter(x='경도', y='위도', grid=True, figsize=(12, 8))
plt.show()

주소 전처리 => 도로명 주소가 NaN이면 지번 주소로 채운다.

In [ ]:
# 도로명 주소가 NaN인 데이터의 개수 => 지번 주소만 입력된 데이터의 개수
# 도로명 주소만 입력되고 지번 주소가 입력되지 않은 데이터는 도로명 주소 제도가 실행된 후 조성된 공원이다.
park_202308.소재지도로명주소.isnull().sum()

In [ ]:
# 도로명 주소는 입력되지 않고 지번 주소만 입력된 데이터
# 불린 인덱싱 작업시 and 조건이나 or 조건이 사용되면 그 조건에 참여하는 데이터에 관계연산자($, |)가 사용되면 연산식 전체를 괄호로 묶어준다.
# park_202308[(park_202308.소재지도로명주소.isnull() == True) & (park_202308.소재지지번주소.notnull() == True)]
# and, or 연산에 참여하는 데이터 자체가 논리값이면 괄호로 묶지 않아도 된다.
park_202308[park_202308.소재지도로명주소.isnull() & park_202308.소재지지번주소.notnull()]

In [ ]:
# 지번 주소가 NaN인 데이터의 개수 => 도로명 주소만 입력된 데이터의 개수
park_202308.소재지지번주소.isnull().sum()

In [ ]:
# 도로명 주소는 입력되고 지번 주소만 입력되지 않은 데이터
park_202308[park_202308.소재지지번주소.isnull() & park_202308.소재지도로명주소.notnull()]

In [ ]:
# 도로명 주소와 지번 주소가 모두 입력된 데이터
park_202308[park_202308.소재지지번주소.notnull() & park_202308.소재지도로명주소.notnull()]

In [ ]:
# 도로명 주소와 지번 주소가 모두 입력되지 않은 데이터
park_202308[park_202308.소재지지번주소.isnull() & park_202308.소재지도로명주소.isnull()]

In [ ]:
# 도로명 주소에 입력된 '-'을(경상북도 칠곡군) 누락값으로 변경한다.
park_202308[park_202308.소재지도로명주소 == '-']

In [ ]:
park_202308.loc[park_202308.소재지도로명주소 == '-', '소재지도로명주소'] = np.NaN # 여기서 칠곡군의 '-' 부분을 NaN으로 바꾸고

In [ ]:
park_202308[park_202308.소재지도로명주소 == '-'] # 다시 검색해보면 없어짐.

In [ ]:
park_202308[park_202308.제공기관명 == '경상북도 칠곡군']

In [ ]:
# 도로명 주소가 NaN인 데이터를 지번 주소로 채운다.
park_202308.소재지도로명주소.fillna(park_202308.소재지지번주소, inplace=True)
park_202308.소재지도로명주소.isnull().sum() # 소재지도로명주소의 null의 개수를 센다.

In [ ]:
park_202308[park_202308.제공기관코드 == '5220000']

CircleMarker를 표시하기 위해 '공원면적' 열에 적당한 수식을 실행해서 '공원면적비율' 열을 만든다.  

In [ ]:
# 시리즈에 저장된 데이터가 lambda 뒤의 변수로 한 건씩 전달되고 수식을 실행한 수가 apply() 메소드에 의해 지정된 시리즈에 일괄 적용된다.
# sqrt(x) * 0.01를 실행하여 면적의 비율을 낮출 수 있다.
# 시리즈.apply(lambda 변수: 수식)
park_202308['공원면적비율'] = park_202308.공원면적.apply(lambda x: np.sqrt(x) * 0.01)
park_202308

도로명 주소에서 공백을 경계로 분리해서 4번째 레벨까지 추가한다.

In [ ]:
park_202308.소재지도로명주소.str.split(' ') # 여기까지는 리스트가 들어있는 시리즈 완성.

In [ ]:
# split() 메소드 실행시 expand=True 옵션을 지정하면 구분자를 경계로 분리된 결과가 데이터프레임으로 만들어진다.
park_202308.소재지도로명주소.str.split(' ', expand=True)   # 데이터프레임 완성

In [ ]:
#  도로명 주소에서 공백으로 분리한 주소를 4번째 레벨까지 추가한다.
park_202308['주소1'] = park_202308.소재지도로명주소.str.split(' ', expand=True)[0]
park_202308['주소2'] = park_202308.소재지도로명주소.str.split(' ', expand=True)[1]
park_202308['주소3'] = park_202308.소재지도로명주소.str.split(' ', expand=True)[2]
park_202308['주소4'] = park_202308.소재지도로명주소.str.split(' ', expand=True)[3]
park_202308    # 소재지도로명주소를 split으로 분리해서 열 추가

위도, 경도 전처리 => 대한민국 영토를 벗어난 위도, 경도와 입력되지 않은 위도, 경도를 찾아 처리한다.

In [ ]:
park_202308[['위도','경도']].describe()

In [ ]:
# 위도와 경도가 대한민국 영토 범위를 벗어난 데이터를 추출한다.
park_202308_error1 = park_202308[(park_202308.위도 < 32) | (park_202308.경도 > 132) ] # 관계연산자 쓰려면 괄호로 묶어줌.
park_202308_error1
# 제24호소공원: 37.37288884,126.95538786
# 근린공원5(만수공원): 36.63146536,127.33220397

In [ ]:
# 위도와 경도가 입력되지 않은 데이터를 추출한다.
park_202308_error2 = park_202308[park_202308.위도.isnull() | park_202308.경도.isnull()]
park_202308_error2
# (신청사)어린이공원 : 36.07569985,126.70329536
# (신청사) 소공원 : 36.07577074,126.70104727

In [ ]:
# 위도와 경도가 올바른 데이터만 추출한다.
park_202308_ok = park_202308[(park_202308.위도 >= 32) & (park_202308.경도 <= 132)]
park_202308_ok

In [ ]:
park_202308_ok.주소1.value_counts()

In [ ]:
# 주소1 컬럼이 '대전'인 데이터를 '대전광역시'로 수정한다.
park_202308_ok.loc[park_202308_ok.주소1 =='대전', '주소1'] = '대전광역시'
# 주소1 컬럼이 '강원도'인 데이터를 '강원특별자치도'로 수정한다.
park_202308_ok.loc[park_202308_ok.주소1 =='강원도', '주소1'] = '강원특별자치도'

In [ ]:
park_202308_ok.주소1.value_counts()

시도별 공원 데이터 시각화

In [ ]:
# ggplot으로 지도 찍기
(
    ggplot(park_202308_ok, aes(x='경도', y='위도', color='주소1'))
        +geom_point()
        +theme(text=element_text(family='NanumGothicCoding'), figure_size=(6, 8))
)

In [ ]:
# seaborn으로 지도 찍기
plt.figure(figsize=(8, 11))
sns.scatterplot(data=park_202308_ok, x='경도', y='위도', hue='주소1', s=15)
plt.legend(loc=1, bbox_to_anchor=(1.4, 0.7))  # 범례위치 loc는 박스 안에서, bbox는 박스 밖으로 내보내기
plt.show()

공원 구분별 분포현황

In [ ]:
park_202308_ok.공원구분.value_counts()

In [ ]:
# 공원구분 컬럼이 '기타공원'인 데이터를 '기타'로 수정한다.
park_202308_ok.loc[park_202308_ok.공원구분 =='기타공원', '공원구분'] = '기타'
park_202308_ok.공원구분.value_counts()
# 공원구분 컬럼이 '어린인공원'인 데이터를 '어린이공원'로 수정한다.
park_202308_ok.loc[park_202308_ok.공원구분 =='어린인공원', '공원구분'] = '어린이공원'
park_202308_ok.공원구분.value_counts()
# 공원구분 컬럼이 '도시자연공원구역'인 데이터를 '도시자연공원'로 수정한다.
park_202308_ok.loc[park_202308_ok.공원구분 =='도시자연공원구역', '공원구분'] = '도시자연공원'
park_202308_ok.공원구분.value_counts()

In [ ]:
# ggplot으로 지도 찍기
(
    ggplot(park_202308_ok, aes(x='경도', y='위도', color='공원구분', size='공원면적비율'))  # size  => 범례에 동구라미로 사이즈 하나 더 생김 
        +geom_point()
        +theme(text=element_text(family='NanumGothicCoding'), figure_size=(6, 8))
)

In [ ]:
# seaborn으로 지도 찍기
plt.figure(figsize=(8, 11))
sns.scatterplot(data=park_202308_ok, x='경도', y='위도', hue='공원구분', size='공원면적비율', sizes=(10, 300))   # size => 범례에 동구라미로 사이즈 하나 더 생김. ggplot과 같음.  sizes => 동구라미 크기의 최소최대값을 지정할 수 있다.
plt.legend(loc=1, bbox_to_anchor=(1.4, 0.7))
plt.show()

어린이공원을 제외한 공원 분포 현황

In [ ]:
(
    ggplot(park_202308_ok[park_202308_ok.공원구분 != '어린이공원'], aes(x='경도', y='위도', color='공원구분', size='공원면적비율'))
        +geom_point()
        +theme(text=element_text(family='NanumGothicCoding'), figure_size=(6, 8))
)

In [ ]:
plt.figure(figsize=(8, 11))
sns.scatterplot(data=park_202308_ok[park_202308_ok.공원구분 != '어린이공원'], x='경도', y='위도', hue='공원구분', size='공원면적비율', sizes=(10, 300))  
plt.legend(loc=1, bbox_to_anchor=(1.4, 0.7))
plt.show()

시도별 공원 비율

In [ ]:
# value_counts()메소드는 시도별 개수 또는 전체 개수에 대한 비율을 계산한다.
# value_counts()메소드의 normalize 옵션의 기본값은 False이고, 개수를 계산하고 True로 지정하면 전체 개수에 대한 비율을 계산한다.
# value_counts()메소드의 ascending 옵션의 기본값은 False이고, 내림차순으로 정렬하고 True로 지정하면 오름차순으로 정렬한다.
# value_counts()메소드의 dropna 옵션은 기본값이 Ture이고, NaN를 무시하고 False로 변경하면 NaN을 포함시켜 계산한다.
park_sum = pd.DataFrame(park_202308_ok.주소1.value_counts())
park_sum.head()

In [ ]:
park_per = pd.DataFrame(park_202308_ok.주소1.value_counts(normalize=True)) # True로 지정했으므로 비율
park_per.head()

In [ ]:
# 시도별 개수 데이터와 전체 개수에 대한 비율 데이터를 병합한다.
# 병합하려는 데이터프레임에 병합 기준을 인덱스로 지정하려면 left_index 옵션과 right_index 옵션을 각각 True로 지정한다.
park_sido = park_sum.merge(park_per, left_index=True, right_index=True).reset_index() # reset_index() 메소드는 인덱스를 따로 빼줌.
# 병합된 데이터프레임의 열 이름을 변경한다.
park_sido.columns = ['시도', '개수', '비율']
park_sido.head()

In [ ]:
park_sum = pd.DataFrame(park_202308_ok.주소1.value_counts()).reset_index()
park_sum.columns = ['시도', '개수']
park_sum.head()

In [ ]:
park_per = pd.DataFrame(park_202308_ok.주소1.value_counts(normalize=True)).reset_index()
park_per.columns = ['시도', '비율']
park_per.head()

In [ ]:
park_sido = park_sum.merge(park_per, left_on='시도', right_on='시도')
# 시도에 대한 내림차순으로 정렬하고 데이터프레임을 즉시 반영한다.   
# sort_index 인덱스를 오름이나 내림으로. sort_values는 데이터를 정렬. 정렬된 결과를 볼수만 있다. 출력해서도 보려면 변수를 넣어서 만들거나 inplace=True를 지정해야함.
# park_sido.sort_values('시도', ascending=False, inplace=True)   
park_sido

In [ ]:
(
    ggplot(park_sido, aes(x='시도', y='개수'))
        + geom_bar(stat='identity', position='dodge', fill='green')
        + theme(text=element_text(family='NanumGothicCoding'), figure_size=(14,8), axis_text_x=element_text(rotation=45))
)

In [ ]:
plt.figure(figsize=(20, 8))
ax = sns.barplot(data=park_sido, x='시도', y='개수')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
from plotnine import *
from plotnine.data import mpg

In [ ]:
(ggplot(mpg)
 + aes(x='manufacturer')
 + geom_bar(size=20)
 + coord_flip() # 세로 막대 그래프를 만든다.
 + labs(y='Count', x='Manufacturer', title='Number of Cars by Make')
)

In [ ]:
park_list = park_202308_ok['주소1'].value_counts().index.tolist()
parkList = pd.Categorical(park_202308_ok['주소1'], categories=park_list)
park_202308_ok = park_202308_ok.assign(공원목록 = parkList)
park_202308_ok

(
    ggplot(park_202308_ok)
    + aes(x='공원목록')
    + geom_bar(size=20, fill= 'pink')
    #+ coord_flip()
    + labs(y='공원개수', x='시도', title='시도별 공원 개수')
    + theme(text=element_text(family='NanumGothicCoding'), figure_size=(14,8), axis_text_x=element_text(rotation=45))
)

공원 구분별 개수

In [ ]:
park_type = pd.DataFrame(park_202308_ok.공원구분.value_counts()).reset_index()
park_type.columns = ['공원구분', '개수']
park_type

In [ ]:
(
    ggplot(park_type, aes(x='공원구분', y='개수'))
    + geom_bar(stat='identity', position='dodge', fill='green')
    + coord_flip() # 차트의 x축 y축을 회전시킨다.
    + labs(y='공원개수', x='시도', title='시도별 공원 개수')
    + theme(text=element_text(family='NanumGothicCoding'), figure_size=(14, 8))
)

In [ ]:
plt.figure(figsize=(20, 8))
ax = sns.barplot(data=park_type, x='공원구분', y='개수')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.show()

경기도 공원 분포

In [ ]:
gyeonggi = park_202308_ok[park_202308_ok.주소1 == '경기도']
gyeonggi

In [ ]:
gyeonggi_type = pd.DataFrame(gyeonggi['공원구분'].value_counts()).reset_index()
park_type.columns = ['공원구분', '개수']
park_type

In [ ]:
gyeonggi_map = folium.Map(location=[gyeonggi.위도.mean(), gyeonggi.경도.mean()], zoom_start=10)
for index, data in gyeonggi.iterrows():
    parkName = folium.Popup('{}: {}'.format(data.공원명, data.소재지도로명주소), max_width=300)
    folium.Marker(location=[data.위도, data.경도], popup=parkName).add_to(gyeonggi_map)
gyeonggi_map

수원시 공원 분포

In [ ]:
suwon = park_202308_ok[park_202308_ok.주소2 == '수원시']
suwon

In [ ]:
suwon_map = folium.Map(location=[suwon.위도.mean(), suwon.경도.mean()], zoom_start=13)
for index, data in gyeonggi.iterrows():
    parkName = folium.Popup('{}: {}'.format(data.공원명, data.소재지도로명주소), max_width=300)
    folium.Marker(location=[data.위도, data.경도], popup=parkName).add_to(suwon_map)
suwon_map

In [ ]:
suwon_map = folium.Map(location=[suwon.위도.mean(), suwon.경도.mean()], zoom_start=13)
for index, data in gyeonggi.iterrows():
    parkName = folium.Popup('{}: {}'.format(data.공원명, data.소재지도로명주소), max_width=300)
    folium.CircleMarker(location=[data.위도, data.경도], popup=parkName,
                       radius=data.공원면적비율, color='red', fill_color='white').add_to(suwon_map)
suwon_map

경기도 일부 공원만 보기

In [ ]:
import re

In [ ]:
park_type = r'.*([역사|체육|수변|문화|묘지]공원).*'
gyeonggi_sample = gyeonggi[gyeonggi.공원구분.str.match(park_type)]
gyeonggi_sample

In [ ]:
set(gyeonggi_sample.공원구분)

In [ ]:
gyeonggi_sample_map = folium.Map(location=[gyeonggi_sample.위도.mean(), gyeonggi_sample.경도.mean()], zoom_start=10)
for index, data in gyeonggi_sample.iterrows():
    parkName = folium.Popup('{}: {}'.format(data.공원명, data.소재지도로명주소), max_width=300)
    folium.Marker(location=[data.위도, data.경도], popup=parkName).add_to(gyeonggi_sample_map)
gyeonggi_sample_map

In [ ]:
gyeonggi_sample_map = folium.Map(location=[gyeonggi_sample.위도.mean(), gyeonggi_sample.경도.mean()], zoom_start=10)
for index, data in gyeonggi_sample.iterrows():
    parkName = folium.Popup('{}: {}'.format(data.공원명, data.소재지도로명주소), max_width=300)
    folium.CircleMarker(location=[data.위도, data.경도], popup=parkName,
                       radius=data.공원면적비율, color='red', fill_color='white').add_to(gyeonggi_sample_map)
gyeonggi_sample_map

서울 공원 분포

In [ ]:
seoul = park_202308_ok[park_202308_ok.주소1 == '서울특별시']
seoul